In [1]:
import tensorflow as tf
import keras 
from keras.applications import DenseNet201
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, NumpyArrayIterator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import confusion_matrix
from keras import models, layers, optimizers, Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D

Using TensorFlow backend.


In [2]:
conv_base = DenseNet201(weights='imagenet', include_top = False, input_shape=(300,300,3))

In [5]:
from google.colab import drive
drive.mount("/content/gdrive/")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [3]:
X = np.load("gdrive/My Drive/pcb/xtrain.npy")
y = np.load("gdrive/My Drive/pcb/ytrain.npy")

In [ ]:
conv_base.predict(X[0].reshape(1,300,300,3)).shape

(1, 9, 9, 1920)

In [ ]:
# Training set has 472 samples

Training with k-fold Validation

In [ ]:
X.shape, (y == 0).sum(), (y == 1).sum()

((472, 300, 300, 3), 236, 236)

In [4]:
datagen = ImageDataGenerator(
    featurewise_center = True,
    featurewise_std_normalization = True
)


datagen.fit(X)

In [5]:
nos = X.shape[0]
bs = int(nos/8)


numit = NumpyArrayIterator(
    X, y, datagen, batch_size=bs, shuffle=False, sample_weight=None,
    seed=None, data_format=None, save_to_dir=None, save_prefix='',
    save_format='png', subset=None, dtype=None
)

samples = np.zeros(shape=(nos,9,9,1920))
labels = np.zeros(shape=(nos))
i = 0
for samples_batch,labels_batch in numit:
    f = conv_base.predict(samples_batch)
    print(f.shape)
    samples[i*bs:(i+1)*bs] = f
    labels[i*bs:(i+1)*bs] = labels_batch
    i+=1
    if (i*bs)>=nos:
        break

(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)


In [ ]:
samples.shape, labels.shape

((472, 155520), (472,))

In [6]:
# To save memory
del X
del y

In [7]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=8)

accuracies = []
f = 1

for train_index, test_index in skf.split(samples, labels):
        X_train = samples[train_index]
        X_test = samples[test_index]
        y_train = labels[train_index]
        y_test = labels[test_index]
      
        model = models.Sequential()
        model.add(layers.Conv2D(64, (2, 2), activation='relu',input_shape=(9, 9, 1920)))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Conv2D(256, (2, 2), activation='relu'))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Flatten())
        model.add(layers.Dense(128, activation='relu'))
        model.add(layers.Dense(64, activation='relu'))
        model.add(layers.Dense(1, activation='sigmoid'))
        model.compile(optimizer = optimizers.Adam(lr = 1e-3),loss = 'binary_crossentropy',metrics = ['accuracy'])
        my_callbacks = [EarlyStopping(monitor = 'val_loss', patience=5, restore_best_weights=True)]

        print('Fold {}'.format(f))
        history = model.fit(X_train, y_train, epochs = 100, batch_size = 20, validation_data = (X_test, y_test), shuffle = True, callbacks = my_callbacks)

        test_preds = model.predict(X_test, batch_size = 20)
        test_predclass = np.array([1.0 if i>0.5 else 0.0 for i in test_preds])

        accuracies.append([confusion_matrix(y_test,test_predclass)[0][0]/(y_test == 0.0).sum(), confusion_matrix(y_test,test_predclass)[1][1]/(y_test == 1.0).sum()])
        f+=1


Fold 1
Train on 413 samples, validate on 59 samples
Epoch 1/100
413/413 [==============================] - 1s 2ms/step - loss: 0.5719 - accuracy: 0.6731 - val_loss: 0.4089 - val_accuracy: 0.8305
Epoch 2/100
413/413 [==============================] - 0s 1ms/step - loss: 0.3109 - accuracy: 0.8741 - val_loss: 0.4493 - val_accuracy: 0.8475
Epoch 3/100
413/413 [==============================] - 0s 1ms/step - loss: 0.1464 - accuracy: 0.9492 - val_loss: 0.3731 - val_accuracy: 0.8644
Epoch 4/100
413/413 [==============================] - 0s 1ms/step - loss: 0.0853 - accuracy: 0.9831 - val_loss: 0.3183 - val_accuracy: 0.8983
Epoch 5/100
413/413 [==============================] - 0s 1ms/step - loss: 0.0899 - accuracy: 0.9685 - val_loss: 0.3455 - val_accuracy: 0.8814
Epoch 6/100
413/413 [==============================] - 0s 1ms/step - loss: 0.0523 - accuracy: 0.9806 - val_loss: 0.4355 - val_accuracy: 0.8814
Epoch 7/100
413/413 [==============================] - 0s 1ms/step - loss: 0.0495 - accura

In [8]:
zero_acc = 0
one_acc = 0

for i in accuracies:
  zero_acc+= i[0]
  one_acc+=i[1]

zero_acc/=8
one_acc/=8

print(zero_acc, one_acc)

0.8385057471264368 0.8224137931034483


Training on whole Training Dataset

In [9]:
model = models.Sequential()
model.add(layers.Conv2D(64, (2, 2), activation='relu',input_shape=(9, 9, 1920)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(256, (2, 2), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

model.compile(optimizer = optimizers.Adam(lr = 1e-3),loss = 'binary_crossentropy',metrics = ['accuracy'])
my_callbacks = [EarlyStopping(monitor = 'val_loss', patience=5, restore_best_weights=True)]


Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 8, 8, 64)          491584    
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 4, 4, 64)          0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 3, 3, 256)         65792     
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 1, 1, 256)         0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 256)               0         
_________________________________________________________________
dense_25 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_26 (Dense)             (None, 64)               

In [12]:
samples.shape

(472, 9, 9, 1920)

In [10]:
xtest = np.load('gdrive/My Drive/pcb/xtest.npy')
ytest = np.load('gdrive/My Drive/pcb/ytest.npy')


In [11]:
datagentest = ImageDataGenerator(
    featurewise_center = True,
    featurewise_std_normalization = True
)

In [12]:
datagentest.fit(xtest)

In [13]:
testnos = ytest.shape[0]

numittest = NumpyArrayIterator(
    xtest, ytest, datagentest, batch_size=testnos, shuffle=False, sample_weight=None,
    seed=None, data_format=None, save_to_dir=None, save_prefix='',
    save_format='png', subset=None, dtype=None
)


test_samples = np.zeros(shape=(testnos,9,9,1920))
test_labels = np.zeros(shape=(testnos))
for samples_batch,labels_batch in numittest:
    g = conv_base.predict(samples_batch)
    print(g.shape)
    test_samples[::] = g
    test_labels[:] = labels_batch
    break

(552, 9, 9, 1920)


In [17]:
# test_samples = test_samples.reshape(testnos,9*9*1920)

In [14]:
historytest = model.fit(samples, labels, epochs = 100, batch_size = 20, validation_data = (test_samples, test_labels), shuffle = True,  callbacks = my_callbacks)

Train on 472 samples, validate on 552 samples
Epoch 1/100
472/472 [==============================] - 1s 3ms/step - loss: 0.5506 - accuracy: 0.6970 - val_loss: 0.4881 - val_accuracy: 0.7572
Epoch 2/100
472/472 [==============================] - 1s 2ms/step - loss: 0.3017 - accuracy: 0.8708 - val_loss: 0.4435 - val_accuracy: 0.8043
Epoch 3/100
472/472 [==============================] - 1s 2ms/step - loss: 0.2094 - accuracy: 0.9174 - val_loss: 0.9518 - val_accuracy: 0.6413
Epoch 4/100
472/472 [==============================] - 1s 2ms/step - loss: 0.1096 - accuracy: 0.9555 - val_loss: 0.6893 - val_accuracy: 0.7699
Epoch 5/100
472/472 [==============================] - 1s 2ms/step - loss: 0.0689 - accuracy: 0.9767 - val_loss: 0.5338 - val_accuracy: 0.8460
Epoch 6/100
472/472 [==============================] - 1s 2ms/step - loss: 0.0570 - accuracy: 0.9788 - val_loss: 0.6153 - val_accuracy: 0.8297
Epoch 7/100
472/472 [==============================] - 1s 2ms/step - loss: 0.0217 - accuracy: 0.

In [15]:
test_preds = model.predict(test_samples, batch_size = 20)

In [16]:
test_predclass = np.array([1.0 if i>0.5 else 0.0 for i in test_preds])

In [17]:
confusion_matrix(test_labels,test_predclass)[0][0]/(ytest == 0.0).sum(), confusion_matrix(test_labels,test_predclass)[1][1]/(ytest == 1.0).sum()

(0.7627118644067796, 0.8093306288032455)

In [ ]:
# Saving model

# model_json = model.to_json()
# with open("gdrive/My Drive/models/VGG19_12.json", "w") as json_file:
#     json_file.write(model_json)

In [ ]:
# model.save_weights("gdrive/My Drive/models/VGG19_12.h5")
# print("Saved model to disk")

In [ ]:
# Saving and loading models
# https://machinelearningmastery.com/save-load-keras-deep-learning-models/

TypeError: ignored